In [16]:
from transformers import AutoTokenizer, pipeline ,BartForConditionalGeneration, BartTokenizer
import torch

model_path = "./Similar_bart_prototype_model"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer =  AutoTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)

BOS=tokenizer.bos_token
EOS =tokenizer.eos_token
PAD = tokenizer.pad_token

replacements = {BOS: "", EOS: "", PAD: ""}

def generate_sequence(context, model, tokenizer, device, num_sequences=2):
      # 모델을 평가 모드로 설정

    # 입력 문맥 토크나이징
    input_ids = tokenizer.encode(BOS + context + EOS, return_tensors="pt").to(device)

    # 모델을 사용하여 출력 생성
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=1024,
        temperature=0.7,
        top_k=5,
        top_p=0.9,
        repetition_penalty=2.0,
        do_sample=True,
        num_return_sequences=num_sequences
    )

    generated_sequences = []
    for generated_sequence in output_sequences:
        # 생성된 시퀀스를 디코딩하여 텍스트로 변환
        decoded_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        generated_sequences.append(decoded_text)
    
    return generated_sequences

def multiple_replace(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

model.eval()
context = "" # 여기에 input값을 넣으세요

# context를 input으로 넣으면 output을 3개 생성함
generated_sentences = generate_sequence(context, model, tokenizer, device, num_sequences=3)

for idx, sentence in enumerate(generated_sentences):
    sentence = multiple_replace(sentence, replacements)
    print(f"생성 {idx+1}:", sentence)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


원본: 그랬더니 해방 후에 또 ○○○○들이 전처럼 드나듭니다.
생성 1: 
생성 2: 
생성 3: 
생성 4: 
생성 5: 
